In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns

# imports best practice sklearn
import sklearn
from sklearn import set_config

# transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# python core library for machine learning and data science
import sklearn
from sklearn import set_config
set_config(transform_output = "pandas")
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from sklearn.datasets import fetch_california_housing


# model selection e hiperparameters optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold, KFold

from sklearn.metrics import accuracy_score
from scipy.spatial import distance
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import set_config
set_config(transform_output = "pandas")
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from sklearn.datasets import fetch_california_housing
from sklearn.impute import KNNImputer

In [ ]:
df_cust_demo = pd.read_csv("/content/drive/MyDrive/TFM/Datasets_merge/df_cust_demo.csv")

In [ ]:
df_sales_prod = pd.read_csv("/content/drive/MyDrive/TFM/Datasets_merge/df_sales_prod.csv")

In [ ]:
df_sales_prod["month_sale"].max()

'2019-05-01'

In [ ]:
df_cust_demo.head()

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,salary
0,1375586,2018-01,2018-01,60,1,2,0,0,0,0,...,0,0,0,0,1,1,29,1,35,87218.10
1,1050611,2018-01,2015-08,57,0,3,0,0,0,0,...,0,0,0,0,1,1,13,0,23,35548.74
2,1050612,2018-01,2015-08,57,0,3,0,0,0,0,...,0,0,0,0,1,1,13,0,23,122179.11
3,1050613,2018-01,2015-08,56,0,3,1,0,0,0,...,0,0,0,0,0,1,50,1,22,119775.54
4,1050614,2018-01,2015-08,57,1,3,0,0,0,0,...,0,0,0,0,1,1,50,0,23,100737.51


In [ ]:
partition = df_cust_demo["pk_partition"].unique()

In [ ]:
df_cust_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929961 entries, 0 to 5929960
Data columns (total 25 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   pk_partition        object 
 2   entry_date          object 
 3   entry_channel       int64  
 4   active_customer     int64  
 5   segment             int64  
 6   short_term_deposit  int64  
 7   loans               int64  
 8   mortgage            int64  
 9   funds               int64  
 10  securities          int64  
 11  long_term_deposit   int64  
 12  credit_card         int64  
 13  payroll             int64  
 14  pension_plan        int64  
 15  payroll_account     int64  
 16  emc_account         int64  
 17  debit_card          int64  
 18  em_account_p        int64  
 19  em_acount           int64  
 20  country_id          int64  
 21  region_code         int64  
 22  gender              int64  
 23  age                 int64  
 24  salary              floa

In [ ]:
df_cust_demo["pk_partition"].max()

'2019-05'

In [ ]:
def transform_datetime_column(df, column_name):
    # Convertir la columna al formato datetime
    df[column_name] = pd.to_datetime(df[column_name])
    # Crear nuevas columnas de 'año' y 'mes'
    df[f'{column_name}_year'] = df[column_name].dt.year
    df[f'{column_name}_month'] = df[column_name].dt.month
    # Comprobar si el dataset original contiene información sobre el día
    if df[column_name].dt.day.nunique() > 1:
        df[f'{column_name}_day'] = df[column_name].dt.day
    # Eliminar la columna datetime original
    df.drop(column_name, axis=1, inplace=True)
    return df

df_cust_demo = transform_datetime_column(df_cust_demo,"pk_partition")
df_cust_demo = transform_datetime_column(df_cust_demo,"entry_date")
df_cust_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929961 entries, 0 to 5929960
Data columns (total 27 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   entry_channel       int64  
 2   active_customer     int64  
 3   segment             int64  
 4   short_term_deposit  int64  
 5   loans               int64  
 6   mortgage            int64  
 7   funds               int64  
 8   securities          int64  
 9   long_term_deposit   int64  
 10  credit_card         int64  
 11  payroll             int64  
 12  pension_plan        int64  
 13  payroll_account     int64  
 14  emc_account         int64  
 15  debit_card          int64  
 16  em_account_p        int64  
 17  em_acount           int64  
 18  country_id          int64  
 19  region_code         int64  
 20  gender              int64  
 21  age                 int64  
 22  salary              float64
 23  pk_partition_year   int64  
 24  pk_partition_month  int6

In [ ]:
df_cust_demo_1 = df_cust_demo[(df_cust_demo["pk_partition_year"] == 2019) & (df_cust_demo["pk_partition_month"] == 5)].copy()

In [ ]:
df_cust_demo_1.head()

,pk_cid,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,...,em_acount,country_id,region_code,gender,age,salary,pk_partition_year,pk_partition_month,entry_date_year,entry_date_month
5489389,657826,2,1,2,0,0,0,0,0,0,...,1,1,25,1,44,54493.38,2019,5,2015,5
5489390,657817,68,0,-1,0,0,0,0,0,0,...,0,1,8,0,32,107701.05,2019,5,2019,5
5489391,657986,7,1,2,0,0,0,0,0,0,...,0,1,41,1,39,100993.17,2019,5,2016,2
5489392,657905,17,1,1,0,0,0,0,0,1,...,1,1,28,1,85,154059.09,2019,5,2017,2
5489393,657336,17,1,2,0,0,0,0,0,0,...,1,1,28,0,38,107692.80,2019,5,2019,3


In [ ]:
tipo_productos = {"short_term_deposit": "activos_financieros",
                  "loans": "creditos_prestamos",
                  "mortgage":"creditos_prestamos",
                  "funds":"activos_financieros",
                  "securities":"activos_financieros",
                  "long_term_deposit": "activos_financieros",
                  "credit_card":"creditos_prestamos",
                  'payroll':"cuenta_operaciones",
                  'payroll_account':"cuenta_operaciones",
                  "pension_plan":"activos_financieros",
                  "emc_account":"cuenta_operaciones",
                  "debit_card":"creditos_prestamos",
                  "em_account_p":"cuenta_operaciones",
                  "em_acount":"cuenta_operaciones"}

In [ ]:
df_cust_demo_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440572 entries, 5489389 to 5929960
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   pk_cid              440572 non-null  int64  
 1   entry_channel       440572 non-null  int64  
 2   active_customer     440572 non-null  int64  
 3   segment             440572 non-null  int64  
 4   short_term_deposit  440572 non-null  int64  
 5   loans               440572 non-null  int64  
 6   mortgage            440572 non-null  int64  
 7   funds               440572 non-null  int64  
 8   securities          440572 non-null  int64  
 9   long_term_deposit   440572 non-null  int64  
 10  credit_card         440572 non-null  int64  
 11  payroll             440572 non-null  int64  
 12  pension_plan        440572 non-null  int64  
 13  payroll_account     440572 non-null  int64  
 14  emc_account         440572 non-null  int64  
 15  debit_card          440572 

In [ ]:
def CreacionVariablesProductos(df,lista_productos):
  for new_col in set(lista_productos.values()):
      df[new_col] = 0

# Sumatorio de los valores en nuevas columnas basado en 'tipo_productos'
  for orig_col, new_col in lista_productos.items():
      df[new_col] += df[orig_col]

  return df

In [ ]:
df_cust_demo_1=CreacionVariablesProductos(df_cust_demo_1,tipo_productos)

In [ ]:
total_cuenta_operaciones = df_cust_demo_1['cuenta_operaciones'].sum()
total_activos_financieros = df_cust_demo_1['activos_financieros'].sum()
total_creditos_prestamos = df_cust_demo_1['creditos_prestamos'].sum()

In [ ]:
columnas_eliminar = list(tipo_productos.keys())

In [ ]:
df_cust_demo_1.drop(columns = columnas_eliminar, inplace = True)

In [ ]:
df_cust_demo_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440572 entries, 5489389 to 5929960
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   pk_cid               440572 non-null  int64  
 1   entry_channel        440572 non-null  int64  
 2   active_customer      440572 non-null  int64  
 3   segment              440572 non-null  int64  
 4   country_id           440572 non-null  int64  
 5   region_code          440572 non-null  int64  
 6   gender               440572 non-null  int64  
 7   age                  440572 non-null  int64  
 8   salary               440572 non-null  float64
 9   pk_partition_year    440572 non-null  int64  
 10  pk_partition_month   440572 non-null  int64  
 11  entry_date_year      440572 non-null  int64  
 12  entry_date_month     440572 non-null  int64  
 13  activos_financieros  440572 non-null  int64  
 14  cuenta_operaciones   440572 non-null  int64  
 15  creditos_p

In [ ]:
df_cust_demo_2 = df_cust_demo_1.copy()

In [ ]:
columnas_revisar = list(set(tipo_productos.values()))


In [ ]:
columnas_revisar

['activos_financieros', 'cuenta_operaciones', 'creditos_prestamos']

In [ ]:
suma_columnas = df_cust_demo_2[columnas_revisar].sum(axis = 1)

In [ ]:
df_cust_demo_3 = df_cust_demo_2[suma_columnas != 0]

In [ ]:
df_cust_demo_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331498 entries, 5489389 to 5929960
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   pk_cid               331498 non-null  int64  
 1   entry_channel        331498 non-null  int64  
 2   active_customer      331498 non-null  int64  
 3   segment              331498 non-null  int64  
 4   country_id           331498 non-null  int64  
 5   region_code          331498 non-null  int64  
 6   gender               331498 non-null  int64  
 7   age                  331498 non-null  int64  
 8   salary               331498 non-null  float64
 9   pk_partition_year    331498 non-null  int64  
 10  pk_partition_month   331498 non-null  int64  
 11  entry_date_year      331498 non-null  int64  
 12  entry_date_month     331498 non-null  int64  
 13  activos_financieros  331498 non-null  int64  
 14  cuenta_operaciones   331498 non-null  int64  
 15  creditos_p

In [ ]:
df_cust_demo_3 = df_cust_demo_3.drop(
    ["pk_partition_year", "pk_partition_month"], axis = 1)

In [ ]:
df_cust_demo_3.to_csv('/content/drive/MyDrive/TFM/clusters/df_cust_demo_limpio.csv', index=False)